import torch
import torchvision
import torchvision.transforms as transform

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

train_size = int(0.7 * len(full_trainset))
val_size = len(full_trainset) - train_size 

trainset, valset = random_split(full_trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Kontrollera storlekar
print(f"Training set: {len(trainset)}, Validation set: {len(valset)}, Test set: {len(testset)}")

Files already downloaded and verified


KeyboardInterrupt: 

In [2]:
import torch.nn as nn
import torch.nn.functional as F

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 9, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(9, 24, 3, padding = 1)
        self.fc1 = nn.Linear(24 * 7 * 7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
epochs = 30

for epoch in range(epochs): 
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 200 == 0:  # Printa loss var 200:e batch
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 0.952
[1,   400] loss: 0.962
[2,   200] loss: 0.848
[2,   400] loss: 0.878
[3,   200] loss: 0.749
[3,   400] loss: 0.806
[4,   200] loss: 0.689
[4,   400] loss: 0.703
[5,   200] loss: 0.583
[5,   400] loss: 0.629
[6,   200] loss: 0.505
[6,   400] loss: 0.574
[7,   200] loss: 0.436
[7,   400] loss: 0.503
[8,   200] loss: 0.381
[8,   400] loss: 0.429
[9,   200] loss: 0.319
[9,   400] loss: 0.381
[10,   200] loss: 0.274
[10,   400] loss: 0.327
[11,   200] loss: 0.229
[11,   400] loss: 0.273
[12,   200] loss: 0.197
[12,   400] loss: 0.231
[13,   200] loss: 0.163
[13,   400] loss: 0.222
[14,   200] loss: 0.143
[14,   400] loss: 0.175
[15,   200] loss: 0.128
[15,   400] loss: 0.159
[16,   200] loss: 0.124
[16,   400] loss: 0.167
[17,   200] loss: 0.108
[17,   400] loss: 0.139
[18,   200] loss: 0.084
[18,   400] loss: 0.107
[19,   200] loss: 0.102
[19,   400] loss: 0.128
[20,   200] loss: 0.099
[20,   400] loss: 0.117
[21,   200] loss: 0.086
[21,   400] loss: 0.104
[22,   200

In [10]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

total_correct = sum(correct_pred.values()) 
total_images = sum(total_pred.values())    

total_accuracy = 100 * total_correct / total_images
print(f'Total accuracy: {total_accuracy:.2f} %')

Accuracy for class: plane is 66.5 %
Accuracy for class: car   is 68.2 %
Accuracy for class: bird  is 58.6 %
Accuracy for class: cat   is 40.7 %
Accuracy for class: deer  is 47.6 %
Accuracy for class: dog   is 51.5 %
Accuracy for class: frog  is 70.6 %
Accuracy for class: horse is 65.8 %
Accuracy for class: ship  is 75.7 %
Accuracy for class: truck is 77.8 %
Total accuracy: 62.30 %


In [11]:
# SGD
#Accuracy for class: plane is 73.9 %
#Accuracy for class: car   is 78.0 %
#Accuracy for class: bird  is 54.1 %
#Accuracy for class: cat   is 42.5 %
#Accuracy for class: deer  is 61.2 %
#Accuracy for class: dog   is 49.3 %
#Accuracy for class: frog  is 72.1 %
#Accuracy for class: horse is 77.0 %
#Accuracy for class: ship  is 71.1 %
#Accuracy for class: truck is 71.0 %
#Total accuracy: 65.02 %

# ADAM
#Accuracy for class: plane is 66.5 %
#Accuracy for class: car   is 68.2 %
#Accuracy for class: bird  is 58.6 %
#Accuracy for class: cat   is 40.7 %
#Accuracy for class: deer  is 47.6 %
#Accuracy for class: dog   is 51.5 %
#Accuracy for class: frog  is 70.6 %
#Accuracy for class: horse is 65.8 %
#Accuracy for class: ship  is 75.7 %
#Accuracy for class: truck is 77.8 %
#Total accuracy: 62.30 %

# KONSTIGT DÅ LOSS VAR BETYDLIGT LÄGRE ; OVERFITTING?